In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import keras.applications.inception_v3 as inception
from sklearn.preprocessing import MultiLabelBinarizer

IMSIZE = (299, 299)

PHOTO_PATH = r"\\devmachine\e$\data\katkam-secret-location\katkam-scaled"



Using TensorFlow backend.


In [2]:
cleaned_data = pd.read_csv("cleaned_data.csv")
weather_array = list(map(lambda x:x.split(',') ,cleaned_data["Mapped"].values))
Y = MultiLabelBinarizer().fit_transform(weather_array)
cleaned_data["Y"] = list(Y)
train_test_mask = np.random.rand(len(cleaned_data)) < 0.8
train_data = cleaned_data[train_test_mask]
test_data = cleaned_data[~train_test_mask]
# cleaned_data

In [3]:
# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')
for layer in base_model.layers:
    print(layer.name)

Loaded Inception model
input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_n

In [4]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
# k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(1024, activation='relu')(base_model.get_layer('avg_pool').output)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='sigmoid', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=[predictions])

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])


# Show some debug output
print (model.summary())

print ('Trainable weights')
print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [5]:

#print(val_files)
#np.random.seed()
img_datagen = ImageDataGenerator(rescale=1./255)

def my_load_img(img_path,img_datagen,size):
    img = image.load_img(img_path, target_size=size)
    x = image.img_to_array(img)
    x = img_datagen.random_transform(x)
    x = img_datagen.standardize(x)
    return x
def my_img_generator(df,img_datagen,batch_size):
#     index_array = np.random.permutation(len(files))
    
    index = 0
    count = 0
    img_datas=[]
    img_labels=[]
    while 1:
        # create numpy arrays of input data
        # and labels, from each line in the file
        item = df.iloc[index]
        if count < batch_size:
            img_datas.append(my_load_img(PHOTO_PATH+"/"+item["Filename"],img_datagen,IMSIZE))
            img_labels.append(item["Y"])
            index=(index+1)%df.shape[0]
            count+=1
        else:
            count=0
            yield (np.array(img_datas),np.array(img_labels))
                # yield (np.array(img_datas),np.array(img_bboxes))
#             else:
#                 yield (np.array(img_datas),np.array(one_hot_labels))
            img_datas = []
            img_labels = []
            img_bboxes=[]
#             random.shuffle(files)
            

batch_size=32
# t = next(my_img_generator(train_files,img_datagen,batch_size))

# model.load_weights('catdog_pretrain_nf.h5') 
# train_data
# train_data.shape
my_train_generator = my_img_generator(train_data,img_datagen,batch_size)
my_test_generator = my_img_generator(test_data,img_datagen,batch_size)

# train_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#         r'E:\workshare\Mind\A3\data\catdog',  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')

#test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#         test_dir,  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')
train_data.shape
# print(next(train_generator)[1])
print(next(my_train_generator)[1])

# print(a[1].shape)
# print(a[1])

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]]


In [6]:
model.fit_generator(
        my_train_generator,
        steps_per_epoch = 20,
        epochs = 5,
        validation_data = my_test_generator,
        verbose = 2,
        validation_steps = 5)
model.save_weights('weather.h5') 


Epoch 1/5
12s - loss: 0.5401 - acc: 0.7281 - val_loss: 0.4684 - val_acc: 0.7900
Epoch 2/5
9s - loss: 0.4388 - acc: 0.8209 - val_loss: 0.4085 - val_acc: 0.8150
Epoch 3/5
9s - loss: 0.3909 - acc: 0.8356 - val_loss: 0.3783 - val_acc: 0.8413
Epoch 4/5
8s - loss: 0.4132 - acc: 0.8163 - val_loss: 0.3844 - val_acc: 0.8238
Epoch 5/5
9s - loss: 0.4379 - acc: 0.8156 - val_loss: 0.3999 - val_acc: 0.8100


In [7]:
model.load_weights('weather.h5') 
img_path = PHOTO_PATH+"/"+"katkam-20160617130000.jpg"
img = image.load_img(img_path , target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
# [Clear, Cloudy, Fog, Rain, Snow]

Predicted: [[ 0.16265836  0.77799559  0.08199696  0.4168593   0.05850774]]


In [8]:
cleaned_data.loc[cleaned_data["Mapped"].str.contains("Clear")]
# cleaned_data["Mapped"].str.contains("Rain")

,Date/Time,Weather,Mapped,Filename,Y
0,2016-06-05 07:00:00,Mainly Clear,Clear,katkam-20160605070000.jpg,"[1, 0, 0, 0, 0]"
8,2016-06-06 16:00:00,Mainly Clear,Clear,katkam-20160606160000.jpg,"[1, 0, 0, 0, 0]"
10,2016-06-07 07:00:00,Mainly Clear,Clear,katkam-20160607070000.jpg,"[1, 0, 0, 0, 0]"
11,2016-06-07 10:00:00,Mainly Clear,Clear,katkam-20160607100000.jpg,"[1, 0, 0, 0, 0]"
20,2016-06-08 19:00:00,Mainly Clear,Clear,katkam-20160608190000.jpg,"[1, 0, 0, 0, 0]"
28,2016-06-10 13:00:00,Mainly Clear,Clear,katkam-20160610130000.jpg,"[1, 0, 0, 0, 0]"
30,2016-06-10 19:00:00,Mainly Clear,Clear,katkam-20160610190000.jpg,"[1, 0, 0, 0, 0]"
36,2016-06-11 19:00:00,Mainly Clear,Clear,katkam-20160611190000.jpg,"[1, 0, 0, 0, 0]"
39,2016-06-12 13:00:00,Mainly Clear,Clear,katkam-20160612130000.jpg,"[1, 0, 0, 0, 0]"
49,2016-06-13 19:00:00,Mainly Clear,Clear,katkam-20160613190000.jpg,"[1, 0, 0, 0, 0]"
